In [ ]:
from nlb_tools.nwb_interface import NWBDataset
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import matplotlib as mpl
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedShuffleSplit
import math
import multiprocessing as mp
from scipy.linalg import orth
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import StratifiedShuffleSplit
from numpy.linalg import svd
from Area2_analysis.lr_funcs import principal_angles
from Area2_analysis.lr_funcs import get_sses_pred, get_sses_mean, nans, calc_proj

matplotlib.rc('font', size=18)
figDir = '/Users/sherryan/Desktop/paper/'

In [ ]:
foldername = "~/area2_population_analysis/s1-kinematics/actpas_NWB/"
monkey = "Han_20171207"
filename = foldername + monkey + "_COactpas_TD_offset6.nwb"

# monkey = 'Duncan_20190710'
# filename = foldername + monkey + "_COactpas_offset6.nwb"

dataset_10ms = NWBDataset(filename, split_heldout=False)
dataset_10ms.resample(10) #in 10-ms bin, has to resample first for Duncan
bin_width = dataset_10ms.bin_width
print(bin_width)
dataset_10ms.smooth_spk(40, name='smth_40')

In [ ]:
n_trials = dataset_10ms.trial_info.shape[0]
print(n_trials,'total trials')
n_neurons = dataset_10ms.data.spikes.shape[1]
print(n_neurons,'neurons')

#make dictionary for trial condition (reaching directions) for Stratified CV
dataset = dataset_10ms
active_mask = (dataset.trial_info.ctr_hold_bump==0) & (dataset.trial_info['split'] != 'none')
passive_mask = (dataset.trial_info.ctr_hold_bump==1) & (dataset.trial_info['split'] != 'none')
nan_mask = (np.isnan(dataset.trial_info.ctr_hold_bump)) & (dataset.trial_info['split'] != 'none')
all_mask = (dataset.trial_info['split'] != 'none')

trial_mask = all_mask
valid_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(valid_n_trials,'valid trials')

trial_mask = active_mask
active_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
active_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(active_n_trials,'active trials')

trial_mask = passive_mask
passive_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
passive_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(passive_n_trials,'passive trials')

trial_mask = nan_mask
nan_trials_idx = np.array(dataset.trial_info.loc[trial_mask]['trial_id'])
nan_n_trials = dataset.trial_info.loc[trial_mask].shape[0]
print(nan_n_trials,'reach bump trials')

active_cond_dir_idx = []
passive_cond_dir_idx = []
nan_cond_dir_idx = []
nan_bump_cond_dir_idx = []
for direction in [0,45,90,135,180,225,270,315]:
# for direction in [0,90,180,270]:
    active_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (dataset.trial_info['ctr_hold_bump'] == 0) & \
           (dataset.trial_info['split'] != 'none'))[0])
    passive_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (dataset.trial_info['ctr_hold_bump'] == 1) & \
           (dataset.trial_info['split'] != 'none'))[0])
    nan_cond_dir_idx.append(np.where((dataset.trial_info['cond_dir']%360 == direction) & (np.isnan(dataset.trial_info.ctr_hold_bump)) & \
           (dataset.trial_info['split'] != 'none'))[0])
    nan_bump_cond_dir_idx.append(np.where((dataset.trial_info['bump_dir']%360 == direction) & (np.isnan(dataset.trial_info.ctr_hold_bump)) & \
           (dataset.trial_info['split'] != 'none'))[0])

active_cond_dict = nans([active_n_trials])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(active_cond_dir_idx)):
        if idx in active_cond_dir_idx[cond]:
            active_cond_dict[i] = cond
            break
    i+=1
print(active_cond_dict)
print(len(active_cond_dict))

passive_cond_dict = nans([passive_n_trials])
i = 0
for idx in passive_trials_idx:
    for cond in range(0,len(passive_cond_dir_idx)):
        if idx in passive_cond_dir_idx[cond]:
            passive_cond_dict[i] = cond
            break
    i+=1
print(passive_cond_dict)
print(len(passive_cond_dict))

nan_cond_dict = nans([nan_n_trials])
i = 0
for idx in nan_trials_idx:
    for cond in range(0,len(nan_cond_dir_idx)):
        if idx in nan_cond_dir_idx[cond]:
            nan_cond_dict[i] = cond
            break
    i+=1
print(nan_cond_dict)
print(len(nan_cond_dict))

nan_bump_cond_dict = nans([nan_n_trials])
i = 0
for idx in nan_trials_idx:
    for cond in range(0,len(nan_bump_cond_dir_idx)):
        if idx in nan_bump_cond_dir_idx[cond]:
            nan_bump_cond_dict[i] = cond
            break
    i+=1
print(nan_bump_cond_dict)
print(len(nan_bump_cond_dict))

if monkey == 'Duncan_20190710':
    active_df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range = (-100,100), ignored_trials = ~active_mask)
    del_indices = list(set(active_trials_idx) - set(active_df['trial_id'].unique()))
    print('was',active_n_trials,'active trials')
    active_n_trials = active_n_trials - len(list(set(active_trials_idx) - set(active_df['trial_id'].unique())))
    active_cond_dict_onset = np.delete(active_cond_dict,np.where(np.isin(active_trials_idx, del_indices)))
    print('now',active_n_trials,'active trials')
    print(len(active_cond_dict_onset))

    passive_df = dataset_10ms.make_trial_data(align_field='move_onset_time', align_range = (-100,100), ignored_trials = ~passive_mask)
    del_indices = list(set(passive_trials_idx) - set(passive_df['trial_id'].unique()))
    print('was',passive_n_trials,'passive trials')
    passive_n_trials = passive_n_trials - len(list(set(passive_trials_idx) - set(passive_df['trial_id'].unique())))
    passive_cond_dict = np.delete(passive_cond_dict,np.where(np.isin(passive_trials_idx, del_indices)))
    print('now',passive_n_trials,'passive trials')
    print(len(passive_cond_dict))

active_df = dataset_10ms.make_trial_data(align_field='move_offset_time', align_range = (-100,0), ignored_trials = ~active_mask)
del_indices = list(set(active_trials_idx) - set(active_df['trial_id'].unique()))
print('was',active_n_trials,'active trials')
active_cond_dict_offset = np.delete(active_cond_dict, np.where(np.isin(active_trials_idx, del_indices))[0])
print('now')
print(len(active_cond_dict_offset))
if monkey == 'Duncan_20190710':
    active_cond_dict = active_cond_dict_onset

In [ ]:
def fit_axes(data, trial_mask, trial_cond_dict, cos_signal, sin_signal, N=10):

    # Select subset of trials
    X = data[trial_mask, :]
    cos_y = cos_signal[trial_mask]
    sin_y = sin_signal[trial_mask]
    cond_dict = np.array(trial_cond_dict)[trial_mask]
    n_trials, dim = X.shape

    # Cosine fits 
    X_proc = X.copy()
    y = cos_y.reshape((n_trials, -1, 1))[:, 0, :]

    axes_list_x = np.full((N, y.shape[1], dim), np.nan)
    r2_list_x = np.full((N,), np.nan)
    r2_cv_list_x = np.full((N,), np.nan)

    for i in range(N):
        reg = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)}).fit(X_proc, y)
        axes_list_x[i, :, :] = reg.best_estimator_.coef_
        r2_list_x[i] = reg.best_estimator_.score(X_proc, y)
        weights = reg.best_estimator_.coef_

        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        true_concat = np.full((n_trials, 1), np.nan)
        pred_concat = np.full((n_trials, 1), np.nan)
        trial_save_idx = 0

        for tr, te in skf.split(range(n_trials), cond_dict):
            X_train, X_test = X_proc[tr, :], X_proc[te, :]
            y_train, y_test = y[tr], y[te]
            lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
            lr.fit(X_train, y_train)
            y_pred = lr.predict(X_test)
            n = len(y_test)
            true_concat[trial_save_idx:trial_save_idx+n] = y_test.reshape(-1, 1)
            pred_concat[trial_save_idx:trial_save_idx+n] = y_pred.reshape(-1, 1)
            trial_save_idx += n

        sses = get_sses_pred(true_concat, pred_concat)
        sses_mean = get_sses_mean(true_concat)
        R2 = 1 - np.sum(sses) / np.sum(sses_mean)
        r2_cv_list_x[i] = R2
        # project out the current axis
        X_proc = X_proc - calc_proj(X_proc, weights.T).T

    # Sine fits 
    X_proc = X.copy()
    y = sin_y.reshape((n_trials, -1, 1))[:, 0, :]

    axes_list_y = np.full((N, y.shape[1], dim), np.nan)
    r2_list_y = np.full((N,), np.nan)
    r2_cv_list_y = np.full((N,), np.nan)

    for i in range(N):
        reg = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)}).fit(X_proc, y)
        axes_list_y[i, :, :] = reg.best_estimator_.coef_
        r2_list_y[i] = reg.best_estimator_.score(X_proc, y)
        weights = reg.best_estimator_.coef_

        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        true_concat = np.full((n_trials, 1), np.nan)
        pred_concat = np.full((n_trials, 1), np.nan)
        trial_save_idx = 0

        for tr, te in skf.split(range(n_trials), cond_dict):
            X_train, X_test = X_proc[tr, :], X_proc[te, :]
            y_train, y_test = y[tr], y[te]
            lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-3, 3, 7)})
            lr.fit(X_train, y_train)
            y_pred = lr.predict(X_test)
            n = len(y_test)
            true_concat[trial_save_idx:trial_save_idx+n] = y_test.reshape(-1, 1)
            pred_concat[trial_save_idx:trial_save_idx+n] = y_pred.reshape(-1, 1)
            trial_save_idx += n

        sses = get_sses_pred(true_concat, pred_concat)
        sses_mean = get_sses_mean(true_concat)
        R2 = 1 - np.sum(sses) / np.sum(sses_mean)
        r2_cv_list_y[i] = R2

        X_proc = X_proc - calc_proj(X_proc, weights.T).T

    return (
        axes_list_x.squeeze(), r2_cv_list_x,
        axes_list_y.squeeze(), r2_cv_list_y
    )

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

x_field = 'spikes_smth_40'
# Fix the trial universe to OFFSET-survivors 
offset_df = dataset.make_trial_data(
    align_field='move_offset_time', align_range=(-100, 0), ignored_trials=~active_mask
)
offset_keep_ids = offset_df['trial_id'].unique()
mask_offset_full = dataset.trial_info.trial_id.isin(offset_keep_ids).to_numpy()
n_offset = len(offset_keep_ids)

# Global z-scoring ONCE on (-100, 1500), but only on offset trials
df_all = dataset.make_trial_data(
    align_field='move_onset_time', align_range=(-100, 1500), ignored_trials=~mask_offset_full
)
dim = dataset.data[x_field].shape[1]
X_all = df_all[x_field].to_numpy().reshape((-1, dim))
mean, std = np.nanmean(X_all, axis=0), np.nanstd(X_all, axis=0)
X_all_z = (X_all - mean) / (std )
# pca = PCA(n_components=40)
# pca.fit(X_all)
# pca.fit(X_all_z)
# print('PCA total var explained:',sum(pca.explained_variance_ratio_))
# plt.plot(np.cumsum(pca.explained_variance_ratio_))

def prepare_epoch_data(align_field, align_range):
    df = dataset.make_trial_data(
        align_field=align_field, align_range=align_range, ignored_trials=~mask_offset_full
    )
    trial_ids = df['trial_id'].unique()
    n_trials = len(trial_ids)

    X = df[x_field].to_numpy().reshape((-1, dim))
    X = (X - mean) / (std )  # use the SAME global mean/std from step 1
    X_trials = X.reshape((n_trials, -1, dim))
    X_mean = np.nanmean(X_trials, axis=1)
    # X_mean_pca = pca.transform(X_mean)

    dirs = np.array([
        dataset.trial_info.loc[dataset.trial_info.trial_id == t, 'cond_dir'].item()
        for t in trial_ids
    ])
    cos_sig = np.cos(np.deg2rad(dirs))
    sin_sig = np.sin(np.deg2rad(dirs))
    return X_mean, dirs, cos_sig, sin_sig

# Onset (CD) and Offset (FB), both restricted to offset trial universe
X_onset, dirs_onset, cos_onset, sin_onset  = prepare_epoch_data('move_onset_time',  (-100, 0))
X_offset, dirs_offset, cos_offset, sin_offset = prepare_epoch_data('move_offset_time', (-100, 0))

# both are over the same offset-survivor trials
n_trials = X_offset.shape[0]
conds = dirs_offset.astype(int)  # stratify by direction (same set for both epochs)

# Make 100 shared 50/50 splits over the offset trial universe 
sss = StratifiedShuffleSplit(n_splits=100, test_size=0.5, random_state=0)
splits = [(tr_idx, te_idx) for tr_idx, te_idx in sss.split(np.arange(n_trials), conds)]


In [ ]:
CD, FB, CD2, FB2, FB3 = [], [], [], [],[]

for tr_idx, te_idx in splits:
    mask_tr = np.zeros(n_trials, dtype=bool)
    mask_te = np.zeros(n_trials, dtype=bool)
    mask_tr[tr_idx] = True
    mask_te[te_idx] = True

    # CD subspace from train split
    axes_x_cd, r2_x_cd, axes_y_cd, r2_y_cd = fit_axes(
        X_onset, mask_tr, conds, cos_onset, sin_onset)
    CD_axes = np.vstack((axes_x_cd[r2_x_cd > 0], axes_y_cd[r2_y_cd > 0]))

    # FB subspace from train split
    axes_x_fb, r2_x_fb, axes_y_fb, r2_y_fb = fit_axes(
        X_offset, mask_tr, conds, cos_offset, sin_offset)
    FB_axes = np.vstack((axes_x_fb[r2_x_fb > 0], axes_y_fb[r2_y_fb > 0]))

    # CD2 and FB2 from test split
    axes_x_cd2, r2_x_cd2, axes_y_cd2, r2_y_cd2 = fit_axes(
        X_onset, mask_te, conds, cos_onset, sin_onset)
    CD_axes2 = np.vstack((axes_x_cd2[r2_x_cd2 > 0], axes_y_cd2[r2_y_cd2 > 0]))

    axes_x_fb2, r2_x_fb2, axes_y_fb2, r2_y_fb2 = fit_axes(
        X_offset, mask_te, conds, cos_offset, sin_offset)
    FB_axes2 = np.vstack((axes_x_fb2[r2_x_fb2 > 0], axes_y_fb2[r2_y_fb2 > 0]))

    # FB3 from test split, orthogonal to CD2
    X_offset_cdOut = X_offset - calc_proj(X_offset,CD_axes2.T).T
    axes_x_fb3, r2_x_fb3, axes_y_fb3, r2_y_fb3 = fit_axes(
        X_offset_cdOut, mask_te, conds, cos_offset, sin_offset)
    FB_axes3 = np.vstack((axes_x_fb3[r2_x_fb3 > 0], axes_y_fb3[r2_y_fb3 > 0]))

    CD.append(CD_axes)
    FB.append(FB_axes)
    CD2.append(CD_axes2)
    FB2.append(FB_axes2)
    FB3.append(FB_axes3)


In [ ]:
np.savez(str(monkey)+"_split_distributions",
         CD = CD, FB = FB, CD2 = CD2, FB2 = FB2, FB3 = FB3,splits = splits)

In [ ]:
monkey = "Han_20171207"


data = np.load(str(monkey)+"_split_distributions.npz",allow_pickle=True)
data.files
CD = data['CD']
FB = data['FB']
CD2 = data['CD2']
FB2 = data['FB2']
FB3 = data['FB3']
splits = data['splits']

In [ ]:
np.savez(str(monkey)+"_split_distributions",
         CD = CD, FB = FB, CD2 = CD2, FB2 = FB2, FB3 = FB3,splits = splits)

In [ ]:
def compute_first_angles(listA, listB):
    avg_angles = []
    for A, B in zip(listA, listB):
        angles_rad = principal_angles(A.T, B.T)
        avg_angles.append(np.degrees(angles_rad[0]))
    return np.array(avg_angles)

angles_CD_FB = compute_first_angles(CD, FB)
angles_CD_CD = compute_first_angles(CD, CD2)
angles_FB_FB = compute_first_angles(FB, FB2)
angles_CD_FB3 = compute_first_angles(CD, FB3)

In [ ]:
figDir = '/Users/sherryan/Desktop/paper/'
matplotlib.rc('font', size=18)

In [ ]:
all_data = np.concatenate([angles_CD_CD, angles_CD_FB, angles_CD_FB3, angles_FB_FB])
xmin, xmax = all_data.min(), all_data.max()
bins = np.linspace(xmin, xmax, 30)

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.hist(angles_CD_CD,color='green',alpha=0.5,edgecolor='black',bins = bins, label = '(H0) CD vs CD2')
plt.hist(angles_CD_FB3,color='orange',alpha=0.5,edgecolor='black',bins = bins, label = '(H1) CD vs FB3')
plt.hist(angles_CD_FB,color='blue',alpha=0.5,edgecolor='black', bins = bins, label = 'CD vs FB')
# plt.legend(fontsize=10)
plt.xlabel('First principal angle (deg)')
plt.ylabel('Count')
# plt.savefig(figDir + monkey + '_angle_dist_first_supp.pdf',dpi = 'figure')
plt.xlim([30,90])

plt.text(40, plt.gca().get_ylim()[1] * 0.9,  # Adjust position as needed
         str(np.mean(angles_CD_CD))+'_'+str(np.mean(angles_CD_FB))+'_'+str(np.mean(angles_CD_FB3)), color='black', fontsize=10)
plt.savefig(figDir + monkey + '_angle_dist_first_main.pdf',dpi = 'figure')

plt.hist(angles_FB_FB,color='magenta',alpha=0.5,edgecolor='black',bins = bins, label = '(H0) FB vs FB2')
plt.text(50, plt.gca().get_ylim()[1] * 0.8,  # Adjust position as needed
         str(np.mean(angles_FB_FB)), color='black', fontsize=10)
plt.savefig(figDir + monkey + '_angle_dist_first_supp.pdf',dpi = 'figure')


plt.show()


In [ ]:
def compute_avg_angles(listA, listB):
    avg_angles = []
    for A, B in zip(listA, listB):
        angles_rad = principal_angles(A.T, B.T)
        avg_angles.append(np.degrees(np.mean(angles_rad)))
    return np.array(avg_angles)

angles_CD_FB_avg = compute_avg_angles(CD, FB)
angles_CD_CD_avg = compute_avg_angles(CD, CD2)
angles_FB_FB_avg = compute_avg_angles(FB, FB2)
angles_CD_FB3_avg = compute_avg_angles(CD, FB3)

In [ ]:
all_data = np.concatenate([angles_CD_CD_avg, angles_CD_FB_avg, angles_CD_FB3_avg, angles_FB_FB_avg])
xmin, xmax = all_data.min(), all_data.max()
bins = np.linspace(xmin, xmax, 30)

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.hist(angles_CD_CD_avg,color='green',alpha=0.5,edgecolor='black',bins = bins, label = '(H0) CD vs CD2')
plt.hist(angles_CD_FB3_avg,color='orange',alpha=0.5,edgecolor='black',bins = bins, label = '(H1) CD vs FB3')
plt.hist(angles_CD_FB_avg,color='blue',alpha=0.5,edgecolor='black', bins = bins, label = 'CD vs FB')
# plt.legend(fontsize=10)
plt.xlabel('Average principal angle (deg)')
plt.ylabel('Count')
# plt.savefig(figDir + monkey + '_angle_dist_first_supp.pdf',dpi = 'figure')
plt.xlim([45,90])

plt.text(40, plt.gca().get_ylim()[1] * 0.9,  # Adjust position as needed
         str(np.mean(angles_CD_CD_avg))+'_'+str(np.mean(angles_CD_FB_avg))+'_'+str(np.mean(angles_CD_FB3_avg)), color='black', fontsize=10)
plt.savefig(figDir + monkey + '_angle_dist_avg_main.pdf',dpi = 'figure')

plt.hist(angles_FB_FB_avg,color='magenta',alpha=0.5,edgecolor='black',bins = bins, label = '(H0) FB vs FB2')
plt.text(50, plt.gca().get_ylim()[1] * 0.8,  # Adjust position as needed
         str(np.mean(angles_FB_FB_avg)), color='black', fontsize=10)
plt.savefig(figDir + monkey + '_angle_dist_avg_supp.pdf',dpi = 'figure')


plt.show()

In [ ]:
# fig, ax = plt.subplots()
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)

# plt.hist(angles_CD_CD_avg, color='green', alpha=0.5, edgecolor='black', bins=20, label='(H0) CD vs CD2')
# plt.hist(angles_FB_FB_avg, color='magenta', alpha=0.5, edgecolor='black', bins=20, label='(H0) FB vs FB2')
# plt.hist(angles_CD_FB_avg, color='blue', alpha=0.5, edgecolor='black', bins=20, label='CD vs FB')
# plt.hist(angles_CD_FB3_avg, color='orange', alpha=0.5, edgecolor='black', bins=20, label='(H1) CD vs FB3')

# plt.legend(fontsize=10)
# plt.xlabel('Average principal angle (deg)')
# plt.ylabel('Count')
# plt.show()

In [ ]:
def ds_alignment_index(X_epoch, CD_axes, FB_axes):
    # center trials
    X = X_epoch - X_epoch.mean(axis=0, keepdims=True)

    # orthonormal bases in neuron space
    D_CD, _  = np.linalg.qr(CD_axes.T)   # (n_neurons x k_cd)
    D_FB, _  = np.linalg.qr(FB_axes.T)   # (n_neurons x k_fb)

    # DS component of the epoch activity (project X into CD subspace)
    P_CD = D_CD @ D_CD.T         
    X_CD = X @ P_CD     
    C_CD = np.cov(X_CD, rowvar=False, bias=True)  # make sure it's (n_neurons x n_neurons)

    # variance of CD-space that lies in FB dims
    num = np.trace(D_FB.T @ C_CD @ D_FB)

    # total CD-space variance
    den = np.trace(D_CD.T @ C_CD @ D_CD) 

    return float(num / den)

In [ ]:
splits.shape

In [ ]:
def masks_from_splits(splits, n_trials):
    m_tr_list, m_te_list = [], []
    for tr_idx, te_idx in splits:
        m_tr = np.zeros(n_trials, dtype=bool); m_tr[tr_idx] = True
        m_te = np.zeros(n_trials, dtype=bool); m_te[te_idx] = True
        m_tr_list.append(m_tr); m_te_list.append(m_te)
    return m_tr_list, m_te_list

m_tr_list, m_te_list = masks_from_splits(splits, n_trials)

# DS alignment distributions
A_DS_CD_FB = []
A_DS_CD_CD = []
A_DS_FB_FB = []
A_DS_CD_FB3 = []

for k in range(len(CD)):
    A_DS_CD_FB.append(ds_alignment_index(X_onset[m_tr_list[k]], CD[k], FB[k]))
    A_DS_CD_CD.append(ds_alignment_index(X_onset[m_tr_list[k]], CD[k], CD2[k]))
    A_DS_FB_FB.append(ds_alignment_index(X_offset[m_tr_list[k]], FB[k], FB2[k]))
    A_DS_CD_FB3.append(ds_alignment_index(X_onset[m_tr_list[k]], CD[k], FB3[k]))

A_DS_CD_FB = np.array(A_DS_CD_FB)
A_DS_CD_CD = np.array(A_DS_CD_CD)
A_DS_FB_FB = np.array(A_DS_FB_FB)
A_DS_CD_FB3 = np.array(A_DS_CD_FB3)

In [ ]:
all_data = np.concatenate([A_DS_CD_CD, A_DS_CD_FB, A_DS_CD_FB3, A_DS_FB_FB])
xmin, xmax = all_data.min(), all_data.max()
bins = np.linspace(xmin, xmax, 30)

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.hist(A_DS_CD_CD,color='green',alpha=0.5,edgecolor='black',bins = bins, label = '(H0) CD vs CD2')
plt.hist(A_DS_CD_FB3,color='orange',alpha=0.5,edgecolor='black',bins = bins, label = '(H1) CD vs FB3')
plt.hist(A_DS_CD_FB,color='blue',alpha=0.5,edgecolor='black', bins = bins, label = 'CD vs FB')
# plt.legend(fontsize=10)
plt.xlabel('Alignment index')
plt.ylabel('Count')
# plt.savefig(figDir + monkey + '_angle_dist_first_supp.pdf',dpi = 'figure') 
plt.xlim([0,.8])

plt.text(.2, plt.gca().get_ylim()[1] * 0.9,  # Adjust position as needed
         str(np.mean(A_DS_CD_CD))+'_'+str(np.mean(A_DS_CD_FB))+'_'+str(np.mean(A_DS_CD_FB3)), color='black', fontsize=10)

plt.savefig(figDir + monkey + '_angle_dist_align_main.pdf',dpi = 'figure')

plt.hist(A_DS_FB_FB,color='magenta',alpha=0.5,edgecolor='black',bins = bins, label = '(H0) FB vs FB2')
plt.text(.3, plt.gca().get_ylim()[1] * 0.8,  # Adjust position as needed
         str(np.mean(A_DS_FB_FB)), color='black', fontsize=10)
plt.savefig(figDir + monkey + '_angle_dist_align_supp.pdf',dpi = 'figure')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.spines['right'].set_visible(False); ax.spines['top'].set_visible(False)
plt.hist(A_DS_CD_CD, color='green',  alpha=0.5, edgecolor='black', bins=20, label='(H0) CD vs CD2')
plt.hist(A_DS_FB_FB, color='magenta',alpha=0.5, edgecolor='black', bins=20, label='(H0) FB vs FB2')
plt.hist(A_DS_CD_FB, color='blue',   alpha=0.5, edgecolor='black', bins=20, label='CD vs FB')
plt.hist(A_DS_CD_FB3, color='orange',alpha=0.5, edgecolor='black', bins=20, label='(H1) CD vs FB3')
plt.legend(fontsize=10)
plt.xlabel('Alignment index')
plt.ylabel('Count')
# plt.title('Direction-Selective Alignment')
plt.show()

In [ ]:
# Alec's code;calculate the alignment index using the Elsayed's method.
def subspace_overlap(data,space):
    # data should be N_neurons x N_time, space should be n_components x N_neurons
    return np.sum(np.diag(space@np.cov(data)@space.T))/np.sum(TruncatedSVD(n_components=n_PC).fit(np.cov(data)).singular_values_)

In [ ]:
def _pca_basis(X, k):
    # center per neuron
    X = np.asarray(X, dtype=np.float64)
    Xc = X - X.mean(axis=0, keepdims=True)
    # SVD of centered data gives PCA; covariance eigenvalues = S^2 / N
    U, S, Vt = np.linalg.svd(Xc, full_matrices=False)
    D = Vt[:k].T                         # neurons x k
    lam = (S**2 / Xc.shape[0])[:k]       # top-k eigenvalues
    return D, lam

def alignment_index_prep_to_move(X_prep, X_move, k=10):
    """
    Elsayed alignment index:
      A = Tr(D_move^T C_prep D_move) / sum_{i=1..k} lambda_prep(i)
    where:
      C_prep is covariance of prep epoch,
      D_move are top-k PCs of move epoch (neurons x k),
      lambda_prep are top-k eigenvalues of C_prep.
    """
    # covariance of prep (centered)
    Xp = X_prep - X_prep.mean(0, keepdims=True)
    C_prep = np.cov(Xp, rowvar=False, bias=True)

    # move PCs (neurons x k)
    D_move, _ = _pca_basis(X_move, k)

    # numerator: prep variance captured by move PCs
    num = np.trace(D_move.T @ C_prep @ D_move)

    # denominator: best possible prep variance captured by k PCs
    evals = np.linalg.eigvalsh(C_prep)           # ascending
    denom = np.sum(evals[-k:])

    return float(num / (denom + 1e-12))

def alignment_index_symmetric(X_onset, X_offset, k=10):
    # symmetric version: average of prep→move and move→prep
    A_pm = alignment_index_prep_to_move(X_onset, X_offset, k)
    A_mp = alignment_index_prep_to_move(X_offset, X_onset, k)
    return A_pm, A_mp, 0.5*(A_pm + A_mp)

In [ ]:
A_pm_splits, A_mp_splits, A_sym_splits = [], [], []

for tr_idx, te_idx in splits:
    m_tr = np.zeros(n_trials, dtype=bool); m_tr[tr_idx] = True
    m_te = np.zeros(n_trials, dtype=bool); m_te[te_idx] = True

    A_pm = alignment_index_prep_to_move(X_onset[m_tr],  X_offset[m_te])
    A_mp = alignment_index_prep_to_move(X_offset[m_tr], X_onset[m_te])
    A_sym = 0.5 * (A_pm + A_mp)

    A_pm_splits.append(A_pm)
    A_mp_splits.append(A_mp)
    A_sym_splits.append(A_sym)

A_pm_splits  = np.array(A_pm_splits)
A_mp_splits  = np.array(A_mp_splits)
A_sym_splits = np.array(A_sym_splits)

In [ ]:
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.hist(A_pm_splits,  bins=20, alpha=0.5, edgecolor='black', label='Prep→Move')
plt.hist(A_mp_splits,  bins=20, alpha=0.5, edgecolor='black', label='Move→Prep')
# plt.hist(A_sym_splits, bins=20, alpha=0.5, edgecolor='black', label='Symmetric')

plt.xlabel('Alignment index')                               
plt.ylabel('Count')
plt.title('Elsayed Alignment Index')
plt.legend()
plt.tight_layout()
plt.show()